In [ ]:
import random
import math

class WumpusWorld:
    def __init__(self):
        self.grid_size = 4
        self.agent_location = (0, 0)
        self.agent_direction = 'right'
        self.wumpus_location = self.random_location()
        self.gold_location = self.random_location(exclude=self.agent_location)
        self.pit_locations = self.random_pit_locations()


    def display_board(self):
        print("Agent Direction:", self.agent_direction)
        print("Agent Location:", self.agent_location)
        print("Wumpus Location:", self.wumpus_location)
        print("Gold Location:", self.gold_location)
        print("Pit Locations:", self.pit_locations)
        print("\nBoard:")
        for i in range(self.grid_size):
            for j in range(self.grid_size):
                if (i, j) == self.agent_location:
                    print("A", end=' ')
                elif (i, j) == self.wumpus_location:
                    print("W", end=' ')
                elif (i, j) == self.gold_location:
                    print("G", end=' ')
                elif (i, j) in self.pit_locations:
                    print("P", end=' ')
                else:
                    print("-", end=' ')
            print()

    def random_location(self, exclude=None):
        while True:
            location = (random.randint(0, self.grid_size - 1), random.randint(0, self.grid_size - 1))
            if exclude is None or location != exclude:
                return location

    def random_pit_locations(self):
        pit_locations = set()
        for i in range(self.grid_size):
            for j in range(self.grid_size):
                if (i, j) != self.agent_location and (i, j) != self.gold_location:
                    if random.random() < 0.2:
                        pit_locations.add((i, j))
        return pit_locations

    def get_percepts(self):
        percepts = [0, 0, 0, 0, 0]  # [Stench, Breeze, Glitter, Bump, Scream]
        adjacent_rooms = self.get_adjacent_rooms()
        if self.wumpus_location in adjacent_rooms:
            percepts[0] = 1  # Stench
        for pit_location in self.pit_locations:
            if pit_location in adjacent_rooms:
                percepts[1] = 1  # Breeze
        if self.agent_location == self.gold_location:
            percepts[2] = 1  # Glitter
        return percepts

    def get_adjacent_rooms(self):
        x, y = self.agent_location
        adjacent_rooms = [(x+1, y), (x-1, y), (x, y+1), (x, y-1)]
        return [(i, j) for i, j in adjacent_rooms if 0 <= i < self.grid_size and 0 <= j < self.grid_size]

    def move_forward(self):
        x, y = self.agent_location
        new_x, new_y = x, y
        if self.agent_direction == 'right':
            new_x = x + 1
        elif self.agent_direction == 'left':
            new_x = x - 1
        elif self.agent_direction == 'up':
            new_y = y + 1
        elif self.agent_direction == 'down':
            new_y = y - 1

        if 0 <= new_x < self.grid_size and 0 <= new_y < self.grid_size:
            self.agent_location = (new_x, new_y)
            return True
        else:
            return False  # Bump

    def turn_left(self):
        directions = ['up', 'left', 'down', 'right']
        current_index = directions.index(self.agent_direction)
        self.agent_direction = directions[(current_index - 1) % 4]

    def turn_right(self):
        directions = ['up', 'left', 'down', 'right']
        current_index = directions.index(self.agent_direction)
        self.agent_direction = directions[(current_index + 1) % 4]

    def grab_gold(self):
        if self.agent_location == self.gold_location:
            self.gold_location = None
            return True
        return False

    def shoot(self):
        if self.wumpus_location in self.get_adjacent_rooms():
            self.wumpus_location = None
            return True
        return False

    def game_over(self):
        if self.agent_location == self.gold_location:
            return "Won"
        elif self.agent_location == self.wumpus_location or self.agent_location in self.pit_locations:
            return "Lost"
        else:
            return False

    def get_legal_actions(self):
        return ['left', 'right', 'forward', 'grab', 'shoot']

    def simulate_action(self, action):
        if action == 'left':
            self.turn_left()
        elif action == 'right':
            self.turn_right()
        elif action == 'forward':
            self.move_forward()
        elif action == 'grab':
            self.grab_gold()
        elif action == 'shoot':
            self.shoot()

    def undo_action(self, action):
        if action == 'left':
            self.turn_right()
        elif action == 'right':
            self.turn_left()
        elif action == 'forward':
            self.move_forward()
        elif action == 'grab':
            self.gold_location = self.agent_location
        elif action == 'shoot':
            pass  # Shooting action cannot be undone

    def utility(self):
        if self.agent_location == self.gold_location:
            return 1000
        elif self.agent_location == self.wumpus_location or self.agent_location in self.pit_locations:
            return -1000
        else:
            return 0

In [ ]:


def minimax_ab_pruning(world, depth, alpha, beta, maximizing_player):
    if depth == 0 or world.game_over():
        return world.utility()

    legal_actions = world.get_legal_actions()

    if maximizing_player:
        max_eval = -math.inf
        for action in legal_actions:
            world.simulate_action(action)
            eval = minimax_ab_pruning(world, depth - 1, alpha, beta, False)
            world.undo_action(action)
            max_eval = max(max_eval, eval)
            alpha = max(alpha, eval)
            if beta <= alpha:
                break  # Beta cut-off
        return max_eval
    else:
        min_eval = math.inf
        for action in legal_actions:
            world.simulate_action(action)
            eval = minimax_ab_pruning(world, depth - 1, alpha, beta, True)
            world.undo_action(action)
            min_eval = min(min_eval, eval)
            beta = min(beta, eval)
            if beta <= alpha:
                break  # Alpha cut-off
        return min_eval

def choose_best_action(world, depth):
    legal_actions = world.get_legal_actions()
    best_action = None
    best_eval = -math.inf

    for action in legal_actions:
        world.simulate_action(action)
        eval = minimax_ab_pruning(world, depth - 1, -math.inf, math.inf, False)
        world.undo_action(action)

        if eval > best_eval:
            best_eval = eval
            best_action = action

    return best_action

def autoplay(world, max_actions, depth):
    actions_taken = 0

    while actions_taken < max_actions:
        percepts = world.get_percepts()
        print("Percepts:", percepts)
        world.display_board()

        if actions_taken % 2 == 0:  # AI player's turn
            action = choose_best_action(world, depth)
            print("AI chooses action:", action)
        else:  # Human player's turn
            action = input("Enter action (left, right, forward, grab, shoot, quit): ").lower()

        world.simulate_action(action)
        actions_taken += 1
        result = world.game_over()
        if result:
            print("Game Over! You", result)
            break

if __name__ == "__main__":
    world = WumpusWorld()
    max_actions = 10  # Set the maximum number of actions
    depth = 3  # Depth of the minimax search
    autoplay(world, max_actions, depth)


Percepts: [0, 0, 0, 0, 0]
Agent Direction: right
Agent Location: (0, 0)
Wumpus Location: (0, 2)
Gold Location: (1, 3)
Pit Locations: set()

Board:
A - W - 
- - - G 
- - - - 
- - - - 
AI chooses action: left
Game Over! You Won
